### AUTH

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize(project='ee-USERNAME/PROJECT')

### **Geometries**

In [ ]:
import ee
import folium
from IPython.display import HTML

#Initialize the Earth Engine API
ee.Initialize()

#Define the study area
geometryPINEGULCH = ee.Geometry.Polygon(
        [[[-108.80164742558028, 39.28402267340665],
          [-108.23722481815841, 39.29358857295844],
          [-108.23997140018966, 39.56831366828538],
          [-108.79752755253341, 39.553491364366145]
          ]])

geometryGRIZZLYCREEK = ee.Geometry.Polygon(
        [[[-107.40066503796388, 39.69746161256133],
          [-107.39860510144044, 39.46619798632684],
          [-106.90834020886231, 39.46778823854969],
          [-106.96670507702638, 39.74763415825682],
          [-107.40272497448731, 39.73337830588799]
          ]])

geometryCHERRYCANYON = ee.Geometry.Polygon(
        [[[-103.53011965523325, 37.51499254964799],
          [-103.53011965523325, 37.336135915833296],
          [-103.33785891304575, 37.33176822282621],
          [-103.33785891304575, 37.519349567331446]]]);

geometrySILVERCREEK = ee.Geometry.Polygon(
        [[[-105.29672060223413, 37.6911297494477],
          [-105.29672060223413, 37.35348518722596],
          [-104.96850404949976, 37.35348518722596],
          [-104.96850404949976, 37.6911297494477]]]);
# 2017-07-19

geometryDECKER = ee.Geometry.Polygon(
        [[[-106.02963654395263, 38.479133428395656],
          [-106.02963654395263, 38.38392784402862],
          [-105.91977326270263, 38.38392784402862],
          [-105.91977326270263, 38.479133428395656]]]);

# 2019-09-08

geometryLAKECHRIST = ee.Geometry.Polygon(
        [[[-107.09745152809535, 39.473198422531205],
          [-107.09745152809535, 39.36445604809376],
          [-106.97797520973597, 39.36445604809376],
          [-106.97797520973597, 39.473198422531205]]]);
#2018-07-30

geometryPALATEAU = ee.Geometry.Polygon(
        [[[-108.55952873183622, 37.67620981411429],
          [-108.55952873183622, 37.534774582410634],
          [-108.39198722792997, 37.534774582410634],
          [-108.39198722792997, 37.67620981411429]]]);


geometries_dict = {
    'PineGulch': geometryPINEGULCH,
    'GrizzlyCreek': geometryGRIZZLYCREEK,
    'CherryCanyon': geometryCHERRYCANYON,
    'SilverCreek': geometrySILVERCREEK,
    'Decker': geometryDECKER,
    'LakeChrist': geometryLAKECHRIST,
    'Palateau': geometryPALATEAU
}

geometries = [
    geometryPINEGULCH,
    geometryGRIZZLYCREEK,
    geometryCHERRYCANYON,
    geometrySILVERCREEK,
    geometryDECKER,
    geometryLAKECHRIST,
    geometryPALATEAU
]

geometryPALATEAU.getInfo()

In [ ]:
def get_fire_boundary(event_id):
    dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')
    filtered_dataset = dataset.filter(ee.Filter.eq('Event_ID', event_id))

    if filtered_dataset.size().getInfo() > 0:
        selected_fire = ee.Feature(filtered_dataset.first())
        fire_geometry = selected_fire.geometry()
        geometry_type = fire_geometry.type().getInfo()

        if geometry_type == 'Polygon':
            coordinates = fire_geometry.coordinates().getInfo()
            fire_polygon = ee.Geometry.Polygon(coordinates)
            return fire_polygon
        elif geometry_type == 'GeometryCollection':
            geometries = fire_geometry.geometries().getInfo()
            fire_multipolygon = ee.Geometry.MultiPolygon(geometries)
            return fire_multipolygon
        else:
            print(f"Unsupported geometry type: {geometry_type}")
            return None
    else:
        print(f"No features found with the specified Event_ID: {event_id}")
        return None


event_id_PineGulch = 'CO3933610852620200731'
event_id_GrizzlyCreek = 'CO3957210726620200810'
event_id_CherryCanyon = 'CO3736710345020200520'
event_id_Decker = 'CO3840910600420190908'
event_id_LakeChrist = 'CO3937110704320180703'
event_id_Palateau = 'CO3765810847420180722'
event_id_SilverCreek = 'CO4022310665520180719'


event_id_dict = {
    'PineGulch': 'CO3933610852620200731',
    'GrizzlyCreek': 'CO3957210726620200810',
    'CherryCanyon': 'CO3736710345020200520',
    'Decker': 'CO3840910600420190908',
    'LakeChrist': 'CO3937110704320180703',
    'Palateau': 'CO3765810847420180722',
    'SilverCreek': 'CO4022310665520180719'
}

fire_boundaries = {}

#for fire_name, event_id in event_id_dict.items():
#    fire_boundary = get_fire_boundary(event_id)
#
#    if fire_boundary is not None:
#        fire_boundaries[fire_name] = fire_boundary
#        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
#        print(fire_boundaries[fire_name].getInfo())

for fire_name, polygon in geometries_dict.items():
    #fire_boundary = get_fire_boundary(polygon)

    if polygon is not None:
        fire_boundaries[fire_name] = polygon
        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
        print(fire_boundaries[fire_name].getInfo())

### **DATES**

In [ ]:
fire_dates = {
    'PineGulch': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-06-18',
        'postfire_start': '2020-09-20',
        'postfire_end': '2020-10-28'
    },
    'GrizzlyCreek': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-07-18',
        'postfire_start': '2020-09-06',
        'postfire_end': '2020-11-01'
    },
    'CherryCanyon':{
        'prefire_start': '2020-04-20',
        'prefire_end': '2020-05-15',
        'postfire_start': '2020-07-01',
        'postfire_end': '2020-08-15'
    },
    'SilverCreek': { #Done
        'prefire_start': '2021-06-01',
        'prefire_end': '2021-08-10',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-30'
    },
    'Decker': { #Done
        'prefire_start': '2019-05-30',
        'prefire_end': '2019-07-25',
       'postfire_start': '2020-5-10',
       'postfire_end': '2020-6-20'
    },
    'LakeChrist': { # done
        'prefire_start': '2017-08-01',
        'prefire_end': '2017-10-15',
        'postfire_start': '2018-09-15',
        'postfire_end': '2018-10-30'
    },
    'Palateau': { #Done
        'prefire_start': '2018-05-10',
        'prefire_end': '2018-06-20',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-20'
    }
}

fire_names = [
    'PineGulch',
    'GrizzlyCreek',
    'CherryCanyon',
    'Decker',
    'LakeChrist',
    'Palateau',
    'SilverCreek'
]

### **READ and SPLIT IMAGE**

In [ ]:
import os
from osgeo import gdal
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
from sklearn.model_selection import train_test_split

def split_image_into_patches(image, patch_size, overlap_factor=2):
    height, width = image.shape
    patches = []
    stride = patch_size // overlap_factor
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    return np.array(patches)

#def split_image_into_patches_test(image, patch_size): No overlap for test
def split_image_into_patches_test(image, patch_size):
    height, width = image.shape
    patches = []
    stride = patch_size  #No overlap
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    return np.array(patches)

################################################################

def read_geotiff(file_path):
    dataset = gdal.Open(file_path, gdal.GA_ReadOnly)
    if dataset is None:
        print(f"Failed to open file: {file_path}")
        return None
    if dataset.RasterCount < 1:
        print(f"No raster bands found in file: {file_path}")
        return None
    band = dataset.GetRasterBand(1)
    data = band.ReadAsArray()
    return data

### RESIZE & **NORMALIZE**

In [ ]:
from osgeo import gdal, gdalconst
import os

def resize_image(src_filename, match_filename, dst_filename):
    # Source
    src = gdal.Open(src_filename, gdalconst.GA_ReadOnly)
    src_proj = src.GetProjection()

    # Match
    match_ds = gdal.Open(match_filename, gdalconst.GA_ReadOnly)
    gdal.UseExceptions()
    match_proj = match_ds.GetProjection()
    match_geotrans = match_ds.GetGeoTransform()
    wide = match_ds.RasterXSize
    high = match_ds.RasterYSize

    # Destination
    dst = gdal.GetDriverByName('GTiff').Create(dst_filename, wide, high, 3, gdalconst.GDT_Float32)
    dst.SetGeoTransform(match_geotrans)
    dst.SetProjection(match_proj)

    # Reproject
    gdal.ReprojectImage(src, dst, src_proj, match_proj, gdalconst.GRA_Bilinear)

    # Flush
    del dst


def normalize_image(src_filename, dst_filename, src_range, dst_range=(0, 100)):
    # Source
    src = gdal.Open(src_filename, gdalconst.GA_ReadOnly)

    # Normalize the source data and save it to the destination file
    gdal.Translate(dst_filename, src, scaleParams=[[src_range[0], src_range[1], dst_range[0], dst_range[1]]], outputType=gdalconst.GDT_Float32)

    # Flush
    del src

In [ ]:
fire = fire_names[0]

#src_filename_s1 = f"S1/RATIO/S1_ratio_{fire}_box.tif"
src_filename_s1 = f"S2/S2_dNBR_{fire}.tif"

#normalized_src_filename_s1 = f"S2/Resized_S2/resized_S2_{fire}.tif"
#normalized_src_filename_s1 = f"S2/S2_dNBR_{fire}.tif"
#normalized_src_filename_s1 = f"S1/RATIO/S1_ratio_{fire}.tif"
normalized_src_filename_s1 = f"S1/RATIO/S1_dNBR_{fire}.tif"

dst_filename_s1 = f"/content/drive/MyDrive/Cleaned_Fire_images_TIFF/S2/Resized_S2/TEST/resized_S2_{fire}_test_NEW2.tif"
#dst_filename_s1 = f"/content/drive/MyDrive/Cleaned_Fire_images_TIFF/S1/RATIO/S1_dNBR_{fire}.tif"


resize_image(src_filename_s1, normalized_src_filename_s1, dst_filename_s1)
print(f"{fire} S1 resized to only have grey scale insted of RGB")